In [61]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR, StepLR
from torchvision import transforms
import torchvision

import pandas as pd
import math
import os
from skimage import io, transform
import numpy as np
import sys
#status bar
from tqdm import tqdm

#import exactly in this way to make sure that matplotlib can generate
#a plot without being connected to a display 
#(otherwise _tkinter.TclError: couldn't connect to display localhost:10.0)
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#Dataset and Transforms
from utils.Dataset_And_Transforms import FigrimFillersDataset, Downsampling, ToTensor, SequenceModeling
from utils.Create_Datasets import create_datasets 
    

#Create datasets
train_loader, val_loader, test_loader = create_datasets(batch_size=1, data_transform=transforms.Compose([ToTensor(), Downsampling(10), SequenceModeling()]))

#Model
#input: image and fixation-input together
#expects input of shape (seq_len, batch, input_size), but we have (batch, seq_len, input_size), so arg batch_first True
rnn = nn.RNN(input_size = 40002, hidden_size=10002, num_layers=1, batch_first=True)

optimizer = optim.SGD(rnn.parameters(), lr=0.5)

criterion = nn.CrossEntropyLoss()

for i, example in enumerate(train_loader): #start at index 0
    # get the inputs
    image = example["image"]
    inputs = example["inputs"]
    targets = example["targets"]
    inputs_combined = example["inputs_combined"]
    
    optimizer.zero_grad()
    
    #hidden = torch.zeros(1,1,10002)
    
    output, hidden = rnn(inputs_combined)
    #CrossEntropyLoss with RNN: output should be 
    #[1, number_of_classes, seq_length], while your target should be [1, seq_length].
    #https://discuss.pytorch.org/t/pytorch-lstm-target-dimension-in-calculating-cross-entropy-loss/30398
    #so switch axes from (1,6,10002 to 1,10002,6)
    output = output.permute(0,2,1)
    loss = criterion(output, targets)
    
    #loss = 0
    #for j in range(inputs.size(1)):
    #    output, hidden = rnn(inputs_combined[0,j].view(1,1,-1), hidden)
    #    loss += criterion(output.view(1,-1), targets[0,j].view(-1))
            
    loss.backward()
    
    optimizer.step()
    
    print(loss.item())
    
    if i == 10:
        break


9.076809883117676
9.181554794311523
10.102762222290039
9.249476432800293
9.239387512207031
8.802957534790039
9.195699691772461
9.25375747680664
10.003707885742188
9.066916465759277
9.47988224029541


In [7]:
output, hn = rnn(inputs_combined) #if hidden_state_0 is not specified, it is just initialized as zeros
print(output.size())

torch.Size([1, 5, 10002])


In [8]:
targets.size()
print(inputs.size())
image.size()
inputs_combined.size()

torch.Size([1, 5, 10002])


torch.Size([1, 5, 40002])

In [15]:
targets.size()

torch.Size([1, 7])

In [15]:
c = torch.empty(6,40002)

a = torch.randn(6,10002)
b = torch.randn(30000)

for i in range(a.size(0)):
    c[i] = torch.cat((a[i],b),0)

In [16]:
c.size()

torch.Size([6, 40002])

In [2]:
output.size()

torch.Size([1, 1, 10002])

In [6]:
targets.size()

torch.Size([1, 10])

In [7]:
targets[0,0]

tensor(3648)

In [29]:
output = torch.randn(1,10002)
criterion(output, torch.Tensor([5]).long())

tensor(11.3556)

In [19]:
loss = nn.CrossEntropyLoss()
input = torch.randn(1, 5, requires_grad=True)
target = torch.empty(1, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)

tensor([0])


In [21]:
input.size()

torch.Size([1, 5])

In [22]:
target.size()

torch.Size([1])

In [24]:
torch.Tensor([5]).size()

torch.Size([1])

In [40]:
a = torch.randn(1,7)
a

tensor([[-1.2866,  0.3221,  1.2652, -0.2681, -0.8693,  1.1387,  0.6439]])

In [41]:
a[0]

tensor([-1.2866,  0.3221,  1.2652, -0.2681, -0.8693,  1.1387,  0.6439])

In [45]:
a[0,0].view(-1).size()

torch.Size([1])